In [1]:
import logging
import coloredlogs
import requests
from time import sleep

from psycopg2.extensions import cursor
from psycopg2.extensions import connection
import pandas as pd

In [2]:
logging.basicConfig()
# logger = logging.getLogger("poll-ooni")
# logger.setLevel(logging.DEBUG)
coloredlogs.install()
coloredlogs.install(level='DEBUG')
# coloredlogs.install(level='INFO')

In [3]:
import config
from imp import reload
reload(config)
from config import config
import psycopg2

# connect to the db
my_connection = psycopg2.connect(**config['postgres'])
my_cursor = my_connection.cursor()
logging.info('Connected to database.')

2021-05-31 09:19:20 congratulations root[14452] INFO Connected to database.


In [4]:
import src.ooni.utils as ooni_utils

TypeError: __init__() got an unexpected keyword argument 'cache_dir'

In [5]:
def rollback():
    my_cursor.execute("ROLLBACK")
    my_connection.commit()

# Visualization!

just connect to database to run cells below.

In [6]:
import json

def load (f):
    return json.load(open(f, 'r'))

sampleData = load('../../visualizations/globe/data/sampleData.json')

import country
export country
value

e -> i


countries are CC2s (perfect)

# Get directed data from the db

In [7]:
# rollback()

import pandas as pd

def unique_pairs (target_attribute:str, confirmed_only=True) -> pd.DataFrame:
    '''
    These are unique inputs that are blocked.
    
    We don't want every blocking example - if a site appears once or more, that's enough.
    '''
    my_cursor.execute(f'''
    SELECT DISTINCT input_url, probe_alpha2, {target_attribute} 
    from ooni_web_connectivity_test
    {"where confirmed=True" if confirmed_only else ""}
    ''')
    pairs = my_cursor.fetchall()
    return pd.DataFrame(pairs, columns=['input', 'e', 'i']).dropna()


In [8]:
# rollback()
from funcy import flatten

def create_weights (**kwargs) -> dict:
    # get data
    pairs_tld = unique_pairs('tld_jurisdiction_alpha2', **kwargs)
    pairs_ip = unique_pairs('input_ip_alpha2', **kwargs)
#     attacks = pairs_tld
    attacks = pd.concat([pairs_tld, pairs_ip])
    # find discrete lines of attack
    lines_of_attack = set([(v[0], v[1]) for v in attacks[['e', 'i']].values])
    # find safe/approved CC2s
    cc2_in_our =  set(flatten(attacks[['e', 'i']].values.tolist()))
    cc2_in_their =  set(flatten([(s['e'], s['i']) for s in sampleData]))
    possibly_unsafe_cc2 = cc2_in_our.difference(cc2_in_their)
    # construct weights
    weights = []
    for line in lines_of_attack:
        e, i = line
        # TODO - hack
        if e not in possibly_unsafe_cc2 and i not in possibly_unsafe_cc2 and e != i:
            num_attacks = len(attacks[(attacks['e']==e) & (attacks['i']==i)])
            weight = num_attacks*10000
            weights.append({
                'e': e,
                'i': i,
                'v': weight
            })
    return weights

In [9]:
# my_connection.commit()

In [10]:
weights_confirmed = create_weights(confirmed_only=True)
weights_all = create_weights(confirmed_only=False)

weights = {
    'dataSetKeys': ['confirmed', 'all'],
    'initDataSetKey': 'confirmed',
    'confirmed': weights_confirmed,
    'all': weights_all
}


In [11]:
json.dump(weights, open('../../visualizations/globe/data/attacks.json', 'w'))

# Some basic questions

how many 'bullets' do we see going by?

In [35]:
pairs_tld = unique_pairs('tld_jurisdiction_alpha2', confirmed_only=True)
pairs_tld
# about 418?

,input,e,i
0,http://www.turkpress.co/,SA,CO
1,http://www.anonymsurfen.com/,SA,US
2,http://www.babylon-x.com/,SA,US
3,http://thenationalcampaign.org/,IR,US
4,http://www.ansar.org/,IR,US
...,...,...,...
716,http://www.centcom.mil/,IR,US
717,http://baskino.me/,RU,US
718,http://www.cbsnews.com/,IR,US
719,http://www.hustler.com/,IR,US


about 418/day?

by destination, who's receiving them?

In [38]:
pairs_ip.groupby('i').count().sort_values('e', ascending=False)

NameError: name 'pairs_ip' is not defined

maybe just a big datacenter there...? load balancer? unclear.
unclear what to make of input IP in general...